<span style="color:orange">
 <h2> Databircks-Assignemnt (Bonus-1)  (Date time Transformation)
</span>
  <h5>
    <span style="color:red">
<b>Author: Deepak Goyal <br>
   <a> adeus.azurelib.com </a><br>
   Email at: admin@azurelib.com
</span>
   

<span style="color:red">
<b>Input Data set is attached in the assignment get the file from there.<br>
   
</span>

In [1]:
# Importing all the required data
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Practise').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

In [2]:
df= spark.read.csv("server_logs.csv",header=True)

In [3]:
df.show(5)

+------+-------------------+---------+----------------+---------------+
|log_id|          timestamp|      url|response_time_ms|     user_agent|
+------+-------------------+---------+----------------+---------------+
|   001|2022-04-21T08:31:31|    /home|              68|    Chrome/91.0|
|   002|2022-08-02T18:08:34|/products|             137|    Chrome/91.0|
|   003|2022-09-04T15:13:31|     /faq|             193|    Chrome/91.0|
|   004|2022-12-26T21:20:29|    /home|             120|    Mozilla/5.0|
|   005|2022-05-13T00:24:33|   /login|              86|Safari/605.1.15|
+------+-------------------+---------+----------------+---------------+
only showing top 5 rows



In [4]:
df.printSchema()

root
 |-- log_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_time_ms: string (nullable = true)
 |-- user_agent: string (nullable = true)



<b>Convert timestamp from string to DateTime.

In [5]:
# Write your code using the Spark Function
df1 = df.withColumn("timestamp", to_timestamp(df["timestamp"], 'yyyy-MM-dd\'T\'HH:mm:ss'))

df1.printSchema()

root
 |-- log_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- url: string (nullable = true)
 |-- response_time_ms: string (nullable = true)
 |-- user_agent: string (nullable = true)



<b> Extract the date from timestamp and add it as a new column date?

In [6]:
# Write your code using the Spark Function
df2 = df1.withColumn('Date',to_date(df1.timestamp,'yyyy-MM-dd'))
df2.show(5)

+------+-------------------+---------+----------------+---------------+----------+
|log_id|          timestamp|      url|response_time_ms|     user_agent|      Date|
+------+-------------------+---------+----------------+---------------+----------+
|   001|2022-04-21 08:31:31|    /home|              68|    Chrome/91.0|2022-04-21|
|   002|2022-08-02 18:08:34|/products|             137|    Chrome/91.0|2022-08-02|
|   003|2022-09-04 15:13:31|     /faq|             193|    Chrome/91.0|2022-09-04|
|   004|2022-12-26 21:20:29|    /home|             120|    Mozilla/5.0|2022-12-26|
|   005|2022-05-13 00:24:33|   /login|              86|Safari/605.1.15|2022-05-13|
+------+-------------------+---------+----------------+---------------+----------+
only showing top 5 rows



<b> Find the number of logs each day.

In [7]:
# Write your code using the Spark Function
df3 = df2.groupBy(col('Date')).count()
df3 = df3.withColumnRenamed('count','logs_per_day')
df3.sort(df3.Date).show(5)

+----------+------------+
|      Date|logs_per_day|
+----------+------------+
|2022-01-03|           2|
|2022-01-04|           1|
|2022-01-10|           1|
|2022-01-19|           1|
|2022-01-20|           1|
+----------+------------+
only showing top 5 rows



<b> Extract hour from timestamp and analyze log count per hour.

In [8]:
# Write your code using the Spark Function
df4 = df2.withColumn('hour',hour(df2.timestamp))
log_count_per_hour = df4.groupBy("hour").count().orderBy("hour",ascending=False)
log_count_per_hour.show(5)

+----+-----+
|hour|count|
+----+-----+
|  23|    3|
|  22|    3|
|  21|    4|
|  20|    7|
|  19|    1|
+----+-----+
only showing top 5 rows



<b> Identify the busiest hour of the day based on log count.

In [9]:
# Write your code using the Spark Function
df5 = log_count_per_hour
x = df5.sort(col('count').desc()).first()
print("Busiest Hour:", x[0], "with log count:", x[1])

Busiest Hour: 10 with log count: 10


<b> Convert timestamp to a different string format: 'YYYY/MM/DD HH:mm'.

In [10]:
df1.select(df1.timestamp).show(5)

+-------------------+
|          timestamp|
+-------------------+
|2022-04-21 08:31:31|
|2022-08-02 18:08:34|
|2022-09-04 15:13:31|
|2022-12-26 21:20:29|
|2022-05-13 00:24:33|
+-------------------+
only showing top 5 rows



In [11]:
# Write your code using the Spark Function
df6 = df1.withColumn('new_timestamp',date_format(to_timestamp(df1.timestamp),'yyyy/MM/dd HH:mm'))
df6.show(5)

+------+-------------------+---------+----------------+---------------+----------------+
|log_id|          timestamp|      url|response_time_ms|     user_agent|   new_timestamp|
+------+-------------------+---------+----------------+---------------+----------------+
|   001|2022-04-21 08:31:31|    /home|              68|    Chrome/91.0|2022/04/21 08:31|
|   002|2022-08-02 18:08:34|/products|             137|    Chrome/91.0|2022/08/02 18:08|
|   003|2022-09-04 15:13:31|     /faq|             193|    Chrome/91.0|2022/09/04 15:13|
|   004|2022-12-26 21:20:29|    /home|             120|    Mozilla/5.0|2022/12/26 21:20|
|   005|2022-05-13 00:24:33|   /login|              86|Safari/605.1.15|2022/05/13 00:24|
+------+-------------------+---------+----------------+---------------+----------------+
only showing top 5 rows



<b> Calculate the average response time for each URL.

In [12]:
# Write your code using the Spark Function
df7 = df2.withColumn('response_time_ms',df2.response_time_ms.cast(FloatType()))
df7.groupBy(df.url).agg(round(avg(df7.response_time_ms),2).\
                        alias('average_response_time_for_each_URL')).\
                        sort(col('average_response_time_for_each_URL')).show()

+---------+----------------------------------+
|      url|average_response_time_for_each_URL|
+---------+----------------------------------+
| /contact|                            109.88|
|/register|                            112.08|
|   /about|                            120.17|
|/products|                            124.18|
|   /login|                            124.67|
|    /blog|                            125.58|
|    /home|                            125.73|
|     /faq|                             138.0|
| /support|                            138.44|
|/services|                            149.75|
+---------+----------------------------------+



<b> Filter logs with response time greater than a certain threshold.


In [13]:
# Write your code using the Spark Function
x = float(input('give threshold '))
df8 = df7.filter(df.response_time_ms>x)
df8.show(5)

give threshold 50
+------+-------------------+---------+----------------+---------------+----------+
|log_id|          timestamp|      url|response_time_ms|     user_agent|      Date|
+------+-------------------+---------+----------------+---------------+----------+
|   001|2022-04-21 08:31:31|    /home|            68.0|    Chrome/91.0|2022-04-21|
|   002|2022-08-02 18:08:34|/products|           137.0|    Chrome/91.0|2022-08-02|
|   003|2022-09-04 15:13:31|     /faq|           193.0|    Chrome/91.0|2022-09-04|
|   004|2022-12-26 21:20:29|    /home|           120.0|    Mozilla/5.0|2022-12-26|
|   005|2022-05-13 00:24:33|   /login|            86.0|Safari/605.1.15|2022-05-13|
+------+-------------------+---------+----------------+---------------+----------+
only showing top 5 rows



<b> Find the top 5 most visited URLs.

In [14]:
# Write your code using the Spark Function# Write your code using the Spark Function
df9 = df.groupBy(col('url')).count()
df9.orderBy('count',ascending=False).limit(5).show()

+---------+-----+
|      url|count|
+---------+-----+
| /support|   16|
|   /about|   12|
|    /blog|   12|
|/register|   12|
|/products|   11|
+---------+-----+



<b> Calculate the day of the week for each log and count logs per day of the week.

In [15]:
# Write your code using the Spark Function
df2 = df2.withColumn('day_of_week',dayofweek(col('Date')))
df2.groupBy(df2.day_of_week).count().orderBy('count',ascending=False).show(5)

+-----------+-----+
|day_of_week|count|
+-----------+-----+
|          3|   18|
|          2|   17|
|          7|   16|
|          5|   14|
|          4|   14|
+-----------+-----+
only showing top 5 rows



<b> Identify logs with empty or null timestamp and count them.

In [16]:
# Write your code using the Spark Function
df10=df2.filter(df2.timestamp.isNull())
df10.show()

+------+---------+---+----------------+----------+----+-----------+
|log_id|timestamp|url|response_time_ms|user_agent|Date|day_of_week|
+------+---------+---+----------------+----------+----+-----------+
+------+---------+---+----------------+----------+----+-----------+



<b> Split user_agent into browser name and version and add as new columns.

In [17]:
# Write your code using the Spark Function
df2 = df2.withColumn('browser_name',split(df2.user_agent,'/').getItem(0))\
         .withColumn('version',split(df2.user_agent,'/').getItem(1))
    
#df2 = df2.withColumn('browser_name', split(df2['user_agent'], '/').getItem(0)) \
         #.withColumn('version', split(df2['user_agent'], '/').getItem(1))

# Show the DataFrame
df2.show(5)

+------+-------------------+---------+----------------+---------------+----------+-----------+------------+--------+
|log_id|          timestamp|      url|response_time_ms|     user_agent|      Date|day_of_week|browser_name| version|
+------+-------------------+---------+----------------+---------------+----------+-----------+------------+--------+
|   001|2022-04-21 08:31:31|    /home|              68|    Chrome/91.0|2022-04-21|          5|      Chrome|    91.0|
|   002|2022-08-02 18:08:34|/products|             137|    Chrome/91.0|2022-08-02|          3|      Chrome|    91.0|
|   003|2022-09-04 15:13:31|     /faq|             193|    Chrome/91.0|2022-09-04|          1|      Chrome|    91.0|
|   004|2022-12-26 21:20:29|    /home|             120|    Mozilla/5.0|2022-12-26|          2|     Mozilla|     5.0|
|   005|2022-05-13 00:24:33|   /login|              86|Safari/605.1.15|2022-05-13|          6|      Safari|605.1.15|
+------+-------------------+---------+----------------+---------

<b> Find the first and last log entry for each day

In [18]:
# Write your code using the Spark Function
window = Window.partitionBy('Date').orderBy('timestamp')

first_log_entry = min('timestamp').over(window)
last_log_entry = max('timestamp').over(window)

result =df2.withColumn('first_log_entry', first_log_entry)\
            .withColumn('last_log_entry', last_log_entry)
result.select('Date', 'first_log_entry', 'last_log_entry').distinct().orderBy('Date').show(5)

+----------+-------------------+-------------------+
|      Date|    first_log_entry|     last_log_entry|
+----------+-------------------+-------------------+
|2022-01-03|2022-01-03 01:50:29|2022-01-03 01:50:29|
|2022-01-03|2022-01-03 01:50:29|2022-01-03 16:11:41|
|2022-01-04|2022-01-04 06:21:23|2022-01-04 06:21:23|
|2022-01-10|2022-01-10 13:04:04|2022-01-10 13:04:04|
|2022-01-19|2022-01-19 15:00:05|2022-01-19 15:00:05|
+----------+-------------------+-------------------+
only showing top 5 rows



<b> Extract the month from timestamp and analyze log count per month.

In [19]:
# Write your code using the Spark Function
df2 = df2.withColumn('month',month(df2.Date))
df2.groupBy(col('month')).count().orderBy('month').show()

+-----+-----+
|month|count|
+-----+-----+
|    1|    8|
|    2|   11|
|    3|    5|
|    4|    5|
|    5|    7|
|    6|   12|
|    7|   11|
|    8|    7|
|    9|   10|
|   10|    9|
|   11|    7|
|   12|    8|
+-----+-----+



<b> Determine the average response time per hour of the day.

In [20]:
df2 = df2.withColumn('time',hour(df2.timestamp))\
.withColumn('response_time_ms',df2.response_time_ms.cast(FloatType()))

In [21]:
# Write your code using the Spark Function
df2.groupBy(col('time')).agg(avg(df2.response_time_ms)).orderBy('avg(response_time_ms)').show(2)

+----+---------------------+
|time|avg(response_time_ms)|
+----+---------------------+
|   9|                 70.0|
|   4|                 72.0|
+----+---------------------+
only showing top 2 rows



<b> Create a new column showing the time of day (Morning, Afternoon, Evening, Night) based on timestamp

In [22]:
# Write your code using the Spark Function
df2 = df2.withColumn('time_of_day',
                when((df2.time<=12) & (df2.time>=7) , 'Morning')
                .when((df2.time<=17) & (df2.time>12) , 'Afternoon')
                .when((df2.time<=1) & (df2.time>17) , 'Evening')
                .otherwise('Night'))
df2.select(df2.time_of_day,df2.time).show(3)

+-----------+----+
|time_of_day|time|
+-----------+----+
|    Morning|   8|
|      Night|  18|
|  Afternoon|  15|
+-----------+----+
only showing top 3 rows



<b> Count the number of logs for each user_agent type.

In [23]:
# Write your code using the Spark Function
df2.groupBy([col('user_agent')]).count().show()

+---------------+-----+
|     user_agent|count|
+---------------+-----+
|    Mozilla/5.0|   18|
|    Chrome/91.0|   31|
|  Edge/18.19041|   23|
|Safari/605.1.15|   28|
+---------------+-----+



<b> Find logs where the url contains a specific keyword (e.g., 'contact').

In [24]:
# Write your code using the Spark Function
df2.filter(df.url.like('%contact%')).show(2)

+------+-------------------+--------+----------------+---------------+----------+-----------+------------+--------+-----+----+-----------+
|log_id|          timestamp|     url|response_time_ms|     user_agent|      Date|day_of_week|browser_name| version|month|time|time_of_day|
+------+-------------------+--------+----------------+---------------+----------+-----------+------------+--------+-----+----+-----------+
|   009|2022-10-29 20:47:12|/contact|           177.0|Safari/605.1.15|2022-10-29|          7|      Safari|605.1.15|   10|  20|      Night|
|   013|2022-10-20 23:37:44|/contact|           147.0|    Chrome/91.0|2022-10-20|          5|      Chrome|    91.0|   10|  23|      Night|
+------+-------------------+--------+----------------+---------------+----------+-----------+------------+--------+-----+----+-----------+
only showing top 2 rows



<b> Convert the timestamp to a Unix timestamp format.

In [25]:
# Write your code using the Spark Function
df2 = df2.withColumn('unix_timestamp',unix_timestamp(df2.timestamp,"MM-dd-yyyy HH:mm:ss"))
df2.show(3)

+------+-------------------+---------+----------------+-----------+----------+-----------+------------+-------+-----+----+-----------+--------------+
|log_id|          timestamp|      url|response_time_ms| user_agent|      Date|day_of_week|browser_name|version|month|time|time_of_day|unix_timestamp|
+------+-------------------+---------+----------------+-----------+----------+-----------+------------+-------+-----+----+-----------+--------------+
|   001|2022-04-21 08:31:31|    /home|            68.0|Chrome/91.0|2022-04-21|          5|      Chrome|   91.0|    4|   8|    Morning|    1650510091|
|   002|2022-08-02 18:08:34|/products|           137.0|Chrome/91.0|2022-08-02|          3|      Chrome|   91.0|    8|  18|      Night|    1659443914|
|   003|2022-09-04 15:13:31|     /faq|           193.0|Chrome/91.0|2022-09-04|          1|      Chrome|   91.0|    9|  15|  Afternoon|    1662284611|
+------+-------------------+---------+----------------+-----------+----------+-----------+----------

<b> Calculate the time difference in minutes between consecutive logs.


In [26]:
# Write your code using the Spark Function
ordered_df = df2.select(df2.timestamp).orderBy('timestamp')
lag_window = Window.orderBy('timestamp')
df_with_lag = ordered_df.withColumn('prev_timestamp', lag('timestamp').over(lag_window))

# Calculate time difference in minutes between consecutive logs
df_with_diff = df_with_lag.withColumn('time_diff_minutes', 
                                      (unix_timestamp('timestamp') - unix_timestamp('prev_timestamp')) / 60)
df_with_diff.select('timestamp', 'prev_timestamp', 'time_diff_minutes').show(5)

+-------------------+-------------------+------------------+
|          timestamp|     prev_timestamp| time_diff_minutes|
+-------------------+-------------------+------------------+
|2022-01-03 01:50:29|               NULL|              NULL|
|2022-01-03 16:11:41|2022-01-03 01:50:29|             861.2|
|2022-01-04 06:21:23|2022-01-03 16:11:41|             849.7|
|2022-01-10 13:04:04|2022-01-04 06:21:23| 9042.683333333332|
|2022-01-19 15:00:05|2022-01-10 13:04:04|13076.016666666666|
+-------------------+-------------------+------------------+
only showing top 5 rows

